In [8]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import imgaug.augmenters as iaa


In [9]:
# Define the paths to your real and fake dataset directories
real_dataset_path = 'C:/Users/ADITHA/Documents/Deep Fake Detection/train/Preprocessed/face_detection/real'
fake_dataset_path = 'C:/Users/ADITHA/Documents/Deep Fake Detection/train/Preprocessed/face_detection/fake'


In [10]:
# List all image files in the real and fake dataset directories
real_image_files = [os.path.join(real_dataset_path, f) for f in os.listdir(real_dataset_path) if os.path.isfile(os.path.join(real_dataset_path, f))]
fake_image_files = [os.path.join(fake_dataset_path, f) for f in os.listdir(fake_dataset_path) if os.path.isfile(os.path.join(fake_dataset_path, f))]

# Combine the lists
image_files = real_image_files + fake_image_files

# Function to load an image
def load_image(file_path):
    image = cv2.imread(file_path)
    return image

# Load all images with tqdm progress bar
images = []
for file in tqdm(image_files, desc="Loading Images"):
    images.append(load_image(file))


Loading Images: 100%|██████████████████████████████████████████████████████████| 1672/1672 [00:08<00:00, 189.19it/s]


In [11]:
# Load the pre-trained Haar Cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Function to detect faces in an image
def detect_faces(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

# Detect faces in all images with tqdm progress bar
detected_faces = []
for image in tqdm(images, desc="Detecting Faces"):
    detected_faces.append(detect_faces(image))


Detecting Faces: 100%|██████████████████████████████████████████████████████████| 1672/1672 [01:25<00:00, 19.61it/s]


In [12]:
# Define augmentation sequence
aug = iaa.Sequential([
    iaa.Fliplr(0.5),  # horizontal flips
    iaa.Affine(rotate=(-20, 20)),  # rotation
    iaa.Multiply((0.8, 1.2)),  # change brightness
    iaa.GaussianBlur(sigma=(0, 1.0))  # blur
])

# Directories to save augmented faces
augmented_faces_dir = 'C:/Users/ADITHA/Documents/Deep Fake Detection/train/Preprocessed/augment'
real_augmented_faces_dir = os.path.join(augmented_faces_dir, 'real')
fake_augmented_faces_dir = os.path.join(augmented_faces_dir, 'fake')
os.makedirs(real_augmented_faces_dir, exist_ok=True)
os.makedirs(fake_augmented_faces_dir, exist_ok=True)

# Function to save augmented faces
def save_augmented_faces(image, faces, file_name, category):
    for i, (x, y, w, h) in enumerate(faces):
        face = image[y:y+h, x:x+w]
        face_aug = aug(image=face)
        if category == 'real':
            face_file_name = os.path.join(real_augmented_faces_dir, f'{os.path.splitext(os.path.basename(file_name))[0]}_aug_face_{i}.jpg')
        else:
            face_file_name = os.path.join(fake_augmented_faces_dir, f'{os.path.splitext(os.path.basename(file_name))[0]}_aug_face_{i}.jpg')
        cv2.imwrite(face_file_name, face_aug)

# Augment and save faces for all images with tqdm progress bar
for file_name, image, faces in tqdm(zip(image_files, images, detected_faces), desc="Augmenting and Saving Faces", total=len(image_files)):
    category = 'real' if file_name in real_image_files else 'fake'
    save_augmented_faces(image, faces, file_name, category)


Augmenting and Saving Faces: 100%|██████████████████████████████████████████████| 1672/1672 [00:18<00:00, 89.13it/s]
